<a href="https://colab.research.google.com/github/alexsnowschool/student-research-project/blob/alex-main/learning-notes-and-experiment/tutorial-knowledge-distallation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This is the knowledge distillation tutorial from the official PyTorch website.

In [18]:
print("testing")

testing


In [11]:
!pip list |  grep torch


torch                            2.1.0+cu121
torchaudio                       2.1.0+cu121
torchdata                        0.7.0
torchsummary                     1.5.1
torchtext                        0.16.0
torchvision                      0.16.0+cu121


In [12]:
!git status

Refresh index: 100% (6/6), done.
On branch alex-main
Your branch is up to date with 'origin/alex-main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	learning-notes-and-experiment/requirements.txt

nothing added to commit but untracked files present (use "git add" to track)


/content/drive/MyDrive/student-research-project


learning-notes-and-experiment  LICENSE	meeting_notes  README.md
